In [94]:
from collections import Counter

import numpy as np
import pandas as pd

import scipy.sparse as spr
import pickle

import import_ipynb
from util import write_json, load_json
from evaluate import CustomEvaluator
from ubcf_model import rec

In [95]:
# train_data, validation_data 준비!
from load_data import song_meta
from load_data import train_data
val_data = pd.read_json('data/val.json', encoding='UTF-8', typ = 'frame')

### training 전에 데이터를 준비해보자!

In [96]:
train_data['istrain'] = 1 # 원본 데이터를 훼손하지 않기 위해서!
val_data['istrain'] = 0

n_train = len(train_data)
n_val = len(val_data)

# train + val
plylist = pd.concat([train_data, val_data], ignore_index=True) # 데이터 전처리를 한 번에 하기 위해

# playlist id
plylist["nid"] = range(n_train + n_val) # tain_data id 새로 생성 (load_data에서 확인해본 결과 0 - 115070가 아니여서)

# id <-> nid
plylist_id_nid = dict(zip(plylist["id"],plylist["nid"]))  # id를 가지고 nid를 구하자!
plylist_nid_id = dict(zip(plylist["nid"],plylist["id"])) # nid를 가지고 id를 구하자!

In [97]:
plylist_song = plylist['songs']
song_counter = Counter([sg for sgs in plylist_song for sg in sgs])
song_dict = {x: song_counter[x] for x in song_counter}

song_id_sid = dict()
song_sid_id = dict()
for i, t in enumerate(song_dict):
  song_id_sid[t] = i
  song_sid_id[i] = t

n_songs = len(song_dict)

In [98]:
train['songs_id'] = plylist['songs'].map(lambda x: [song_id_sid.get(s) for s in x if song_id_sid.get(s) != None])

In [99]:
plylist_use=plylist[['istrain','nid','updt_date','songs_id']]
plylist_use.loc[:,'num_songs'] = train_use['songs_id'].map(len) # num_songs의 열을 만들어서 플리에 들어있는 song의 수 저장
plylist_use =train_use.set_index('nid') # nid를 id로 사용!

KeyError: "['songs_id'] not in index"

In [ ]:
plylist_train = plylist_use.iloc[:n_train,:]
plylist_test = plylist_use.iloc[n_train:,:]

In [ ]:
# sample val (300개만 뽑아서)
np.random.seed(33)
n_sample = 300

test = plylist_val.iloc[np.random.choice(range(n_val), n_sample, replace=False),:]

In [ ]:
# row가 playlist(nid)이고 column이 item(sid)인 sparse matrix A를 만들자!

row = np.repeat(range(n_train), plylist_train['num_songs'])
col = [song for songs in plylist_train['songs_id'] for song in songs]
dat = np.repeat(1, plylist_train['num_songs'].sum())
train_songs_A = spr.csr_matrix((dat, (row, col)), shape=(n_train, n_songs))

In [ ]:
# row가 playlist(nid)이고 column이 item(sid or tid)인 sparse matrix A를 만듭니다.
row = np.repeat(range(n_train), plylist_train['num_songs'])
col = [song for songs in plylist_train['songs_id'] for song in songs]
dat = np.repeat(1, plylist_train['num_songs'].sum())
train_songs_A = spr.csr_matrix((dat, (row, col)), shape=(n_train, n_songs))

In [ ]:
train_songs_A_T = train_songs_A.T.tocsr()

In [ ]:
train_songs_A

### Training을 해보자!

In [ ]:
answers = rec(test.index)

In [ ]:
write_json(answers, "results/results.json") # 예측값 저장!

### 모델 평가를 해보자!

In [ ]:
evaluator = CustomEvaluator()
evaluator.evaluate("data/val.json", "data/results/results.json")